In [6]:
from collections import defaultdict, Counter
from copy import copy
from math import exp, sqrt
from random import random, shuffle, choice, randint, uniform

from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.constraints import non_neg, max_norm
import numpy
from numpy import array, mean, ones
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM, multiply, subtract, add, Activation, Lambda, division, dot
from keras.layers import Dense, concatenate, MaxPooling1D, LocallyConnected1D, Reshape
from keras import backend as K
from keras import constraints
from keras.optimizers import Adam, RMSprop, SGD

from utils import generate_student_name
import random

nq = 12
n_traits = 5
tr_len = 100
tt_len = 10


In [7]:
generate_student_name()

'PHUTHY CORE '

In [48]:
from keras import backend as K
from keras.constraints import Constraint
from keras.engine.topology import Layer
from keras import initializers

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''
    def __init__(self, min_w=0, max_w=4):
        self.min_w = min_w
        self.max_w = max_w

    def __call__(self, p):
        return K.clip(p, self.min_w, self.max_w)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'min_w': self.min_w,
                'max_w': self.max_w }


class ProductLayer(Layer):

    def __init__(self, output_dim, kernel_constraint=WeightClip(min_w=0.0, max_w=4.0), **kwargs):
        self.output_dim = output_dim
        super(ProductLayer, self).__init__(**kwargs)
        self.kernel_constraint = constraints.get(kernel_constraint)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(-1,self.output_dim),
                                      initializer=initializers.RandomUniform(minval=0.0, maxval=4.0), #.Constant(value=2.0),
                                      trainable=True,
#                                       constraint=self.kernel_constraint
                                     )
        
        super(ProductLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        print("sk",self.kernel.shape)
        print("x", x.shape)
#         self.kernel = K.clip(self.kernel, self.cmin, self.cmax)
        z = K.concatenate([x,self.kernel], axis=2)
        print("internal z shape:", z.shape)
        p = K.prod(z, axis=2)
#         p = K.clip(p, self.cmin, self.cmax)
#         self.kernel = K.clip(self.kernel, self.cmin, self.cmax)

#         print("internal output shape:", p.shape)
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [9]:
class Question():
    def __init__(self, qix, nt=10):
        self.id = qix
        #self.qvals = [ int(uniform(0,1)>=0.5) for _ in range(nt) ]
        self.beta = randint(0,4)
        print("beta=",self.beta)


In [50]:
def generate_master_model(nq=10, ns=10):
    psi_sel = Input(shape=(ns,), name="user_selector")
    q_sel = Input(shape=(nq,), name="q_selector")
    #th_in = Input(shape=(ns,), name="user_theta")
#     th_in = Lambda(lambda x: K.ones(shape=(ns,))
    #b_in = Input(shape=(nq,))
#     b_in = Lambda(lambda x: K.ones(shape=(nq,))
#     th_in = K.ones(shape=ns)
#     b_in = K.ones(shape=nq)
    th_in = Input(shape=(ns,), name="dummy_th_in")
    b_in = Input(shape=(nq,), name="dummy_b_in")

    # N x ns .
    
    th_seld = multiply([psi_sel, th_in])
    b_seld = multiply([q_sel, b_in])
    
    th_layer = ProductLayer(ns)(th_seld) # introduce a trainable weight
    b_layer = ProductLayer(nq)(b_seld) # introduce a trainable weight
       
    this_th = Lambda(lambda x : K.sum(x))(th_layer)
    this_b = Lambda(lambda x : K.sum(x))(b_layer)
    
    dif = subtract([this_th,this_b])
    Prs = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="Pr_sigmoid")(dif)
            

In [11]:
def generate_student_struct(nt=10, nq=20):
    name = generate_student_name()
    theta = randint(0,4)
    mastery = [0 for _ in range(nq)]
    
    dict = {}
    dict["name"] = name
    dict["theta"] = theta
    dict["pred_theta_sc"] = 2.0
    
    return dict

In [51]:
def attempt_q(student, q: Question, update_weights=True):
    real_theta = student["theta"]
    real_beta = q.beta
    p = calculate_pass_probability(real_theta, real_beta)

    this_att = uniform(0,1)
    if (this_att <= p):
        passed=1
        student["mastery"][q.id] = 1
    else:
        passed=0

#     if update_weights:
#         pred_beta = q.pred_beta.get_weights()
# #     print(pred_beta)
#         pred_theta = student["pred_theta"].get_weights()
#         print("true th:", student["theta"])
#         print("true beta:", q.beta)
        
#         print("pre ptheta:", pred_theta)
#         print("pre pbeta:", pred_beta)
#         obi = numpy.ones(1)
#         print(obi)
#         es = EarlyStopping(monitor="loss", mode="auto")
#         q.model.fit([pred_theta[0][0], obi],[array([passed])], batch_size=1, verbose=0, epochs=10, callbacks=[es])
#         student["model"].fit([obi, pred_beta[0][0]],[array([passed])], batch_size=1, verbose=0, epochs=1, callbacks=[es])
#         print("post ptheta:", student["pred_theta"].get_weights()[0][0])
#         print("post pbeta:", q.pred_beta.get_weights()[0][0])
        
    return passed

In [13]:
def calculate_pass_probability(a, beta):
    z = (a - beta)
    print("z={}".format(z))
    try:
        p_pass = 1.0 / (1.0 + exp(-z))
        print("p_pass={}".format(p_pass))
    except OverflowError:
        p_pass = 0.0
    #print("real p_pass = {}".format(p_pass))
    return p_pass
    

In [14]:
random.seed(666)
master_qs = [Question(qix, n_traits) for qix in range(nq)]
for q in master_qs:
    print("Q:{}, difficulty={:.2f} across 1 components".format(q.id, q.beta))

beta= 3
beta= 3
beta= 3
beta= 2
beta= 4
beta= 0
beta= 4
beta= 4
beta= 2
beta= 0
beta= 0
beta= 1
Q:0, difficulty=3.00 across 1 components
Q:1, difficulty=3.00 across 1 components
Q:2, difficulty=3.00 across 1 components
Q:3, difficulty=2.00 across 1 components
Q:4, difficulty=4.00 across 1 components
Q:5, difficulty=0.00 across 1 components
Q:6, difficulty=4.00 across 1 components
Q:7, difficulty=4.00 across 1 components
Q:8, difficulty=2.00 across 1 components
Q:9, difficulty=0.00 across 1 components
Q:10, difficulty=0.00 across 1 components
Q:11, difficulty=1.00 across 1 components


# Training
This is where sh!t gets real.  We take our tr_len (1000?) students, and iterate over them 100 times to create 100,000 *complete examples* of a student attacking the curriculum.  The questions themselves are attacked in random order: the student has no intelligent guidance throught the material. (Obvious we may wish to provide that guidance at some point in the future.)

Remember, there are only 12 exercises in the curriculum, so if the student is taking 60 or 70 goes to answer them all, that's pretty poor.  But some of these kids are dumb as lumber, so cut them some slack!  They will all get there in the end since by the CMU AFM practice will, eventually, make perfect!

In [32]:
MIN_THETA =0
MAX_THETA =4
def calc_better_theta(curr_theta, attempts):
    eps = 0.1
    sum_spread=0.0
    sum_info = 0.0
    for ix, (psi,q_i,pf) in enumerate(attempts):
        #go through the answered questions, sum up the difference in value vs expected value
        u_i = float( pf ) #cast bool of response to float i.e. {1,0}
        p_i_theta = calculate_pass_probability(curr_theta, q_i.pred_beta_sc)
        print("real/exp",u_i,p_i_theta)
        sum_spread += ( u_i - p_i_theta ) #TODO check this -ve sign
        #Then get the information function, tells us about precision of estimate
        info = p_i_theta * (1.0-p_i_theta)
        sum_info += info
    print("sum_spread=",sum_spread, "scaled_sp=",(sum_spread/sum_info))
    print("total info", sum_info)
    new_theta = curr_theta + (sum_spread / sum_info)
    print("raw new theta=", new_theta)

    if new_theta >= MAX_THETA:
        new_theta = MAX_THETA
        return new_theta
    elif new_theta <= MIN_THETA:
        new_theta = MIN_THETA
        return new_theta
    elif abs(new_theta - curr_theta) < eps:
        return new_theta # we're done, new theta is calc'd
    else:
        return calc_better_theta(new_theta, attempts) # iterate again

def calc_better_beta(curr_beta, attempts):
    eps = 0.1
    sum_spread=0.0
    sum_info = 0.0
    psis, qs, pfs = zip(*attempts)
    for ix,psi in enumerate(psis):
        th = psi["pred_theta_sc"]
        #go through the answered questions, sum up the difference in value vs expected value
        u_i = float( pfs[ix] ) #cast bool of response to float i.e. {1,0}
        p_i = calculate_pass_probability(th, curr_beta)
        print("Q real/exp",u_i,p_i)
        sum_spread += ( u_i - p_i ) #TODO check this -ve sign
        #Then get the information function, tells us about precision of estimate
        info = p_i * (1.0-p_i)
        sum_info += info
    print("Q sum_spread=",sum_spread, "scaled_sp=",(sum_spread/sum_info))
    print("Q total info", sum_info)
    new_beta = curr_beta - (sum_spread / sum_info)
    print("Q raw new beta=", new_beta)

    if new_beta >= MAX_THETA:
        new_beta = MAX_THETA
        return new_beta
    elif new_beta <= MIN_THETA:
        new_beta = MIN_THETA
        return new_beta
    elif abs(new_beta - curr_beta) < eps:
        return new_beta # we're done, new theta is calc'd
    else:
        return calc_better_beta(new_beta, attempts) # iterate again
    


In [52]:
Xs = {}
Tz = {}
ys = {}
alphas = {}
attempts = []
attempts_by_psi = {}
attempts_by_q = {}
no_tr_students = 200

random.seed(666)
psi_list = [ generate_student_struct() for _ in range(no_tr_students)]

mmodel = generate_master_model(nq = len(master_qs), ns=len(psi_list))

user_budget = 10
for run in range(1):
    print("----{}\n".format(run))
    for psi in psi_list:
        ct = 0
        psi["mastery"] = [0 for _ in range(nq)]
        qs = [ix for ix in range(len(master_qs))]
        print("* * * **** USER {}".format(psi["name"]))
        print("* * * * ** THETA {}".format(psi["theta"]))
            
        while(True):
            qix = random.choice(qs)
            q = master_qs[qix]
            passed=False
#             while not passed:
#             print(qix, q.pred_beta)
#             if qix not in Xs:
#                 Xs[qix]=[]
#                 Tz[qix]=[]
#                 ys[qix]=[]
#                 alphas[qix]=[]
            if psi["name"] not in attempts_by_psi:
                attempts_by_psi[psi["name"]]=[]
            
            if q not in attempts_by_q:
                attempts_by_q[q]=[]
    
            #print("attempting {}".format(q.id))
            passed = attempt_q(psi, q, update_weights=False)
            
            tup = (psi, q, passed)
            attempts.append(tup)
            attempts_by_psi[psi["name"]].append(tup)
            attempts_by_q[q].append(tup)
                        
            ct+=1
#             if passed:
                #print("passed")
            qs.remove(qix)

            if qs == []: # or ct>=user_budget:
                print("* ** *QFIN USER {}".format(psi["name"]))
                break

ValueError: initial_value must have a shape specified: Tensor("product_layer_13/random_uniform:0", shape=(?, 200), dtype=float32)

In [ ]:
obi = numpy.ones(1)
# print(obi)
es = EarlyStopping(monitor="loss", mode="auto")

# for _ in range(10):
#     for qn in attempts_by_q:
#         print("training {}, pre est beta = {}".format(qn.id, qn.pred_beta.get_weights()[0][0]))
#         thetas = array([ tup[0]["pred_theta"].get_weights()[0][0] for tup in attempts_by_q[qn]])
#         passed = array([ tup[2] for tup in attempts_by_q[qn] ])     
#         obi = numpy.ones(len(thetas))
#         qn.model.fit([thetas, obi],passed, batch_size=1, verbose=1, epochs=100, callbacks=[es])
#         print("{}, post est beta = {}. real {}".format(qn.id, qn.pred_beta.get_weights()[0][0], qn.beta))

#     for student_obj in psi_list:
#         student = student_obj["name"]
#         print("training {}".format(student))
#         print("Pre est {} has theta: {}".format(student, student_obj["pred_theta"].get_weights()[0][0]))
#         betas = array([ tup[1].pred_beta.get_weights()[0][0] for tup in attempts_by_psi[student] ])
#         passed = array([ tup[2] for tup in attempts_by_psi[student] ]).reshape(-1,1)
#         print(betas.shape)
#         print(passed.shape)
#         obi = numpy.ones(len(betas))
#         student_obj["model"].fit([obi, betas],passed, batch_size=1, verbose=1, epochs=100, callbacks=[es])
#         print("Post est {} has theta: {}, real {}".format(student, student_obj["pred_theta"].get_weights()[0][0], student_obj["theta"]))
    
random.seed(666)
calibration_mode = "NEURAL"
repz = (10 if calibration_mode=="NEURAL" else 1)
es = EarlyStopping(monitor="loss", mode="auto", min_delta=0.001)
student_att_ct = Counter()
pat_ct = 10
b_del_tol = 0.1
th_del_tol = 0.1
last_th_rmse = 1000
last_b_rmse = 1000
freeze_betas = False
freeze_thetas = False
for _ in range(100):
    if calibration_mode=="NEURAL":
        shuffle(master_qs)
        shuffle(psi_list)
        for q in master_qs:
            print(q.id)
            
            q_1hot = numpy.ones(len(master_qs))
            q_1hot[q.id] = 1

            b_del = tol+1
            thetas = [ tup[0]["pred_theta"].get_weights()[0][0] for tup in attempts_by_q[q]]
#             print("got  thetas")
            pfs = [ tup[2] for tup in attempts_by_q[q]]
#             print("got pfs")
            preb = q.pred_beta.get_weights()[0][0]
#             while(b_del > tol):
#             print("fit q")
            if not freeze_betas:
                q.model.fit([array(thetas), numpy.ones(len(thetas))],[array(pfs)], verbose=0, batch_size=1, epochs=100, callbacks=[es])
            postb = q.pred_beta.get_weights()[0][0]      
#                 b_del = abs(postb - preb)
#                 print("bdel",b_del)
            print("q{} true({}), preb {} postb {}".format(q.id, q.beta, preb, postb))

            if not freeze_thetas:
                psi_sublist = [tup[0] for tup in attempts_by_q[q]]
                for _student in psi_sublist:
                    th_del = tol+1
                    betas = [ tup[1].pred_beta.get_weights()[0][0] for tup in attempts_by_psi[_student["name"]]]
                    pfs = [ tup[2] for tup in attempts_by_psi[_student["name"]]]
                    preth= _student["pred_theta"].get_weights()[0][0] 
            #             while(th_del > tol):
            #             print("fit psi")
                    _student["model"].fit([numpy.ones(len(betas)), array(betas)],[array(pfs)], verbose=0, batch_size=1, epochs=100, callbacks=[es])
                    postth= _student["pred_theta"].get_weights()[0][0]
            #                 th_del = abs(postth - preth)
            #                 print("th_Del",th_del)
                    print("s{}: true({}), {} -> {}".format(_student["name"], _student["theta"], preth, postth))

    else:
        for q in master_qs:
            new_beta = calc_better_beta(q.pred_beta_sc, attempts_by_q[q])#[0:student_att_ct[student["name"]]])
            q.pred_beta_sc = new_beta
        for student in psi_list:
            theta = student["pred_theta_sc"]
            new_theta = calc_better_theta(theta, attempts_by_psi[student["name"]])
            student["pred_theta_sc"] = new_theta
            
            
    if calibration_mode=="NEURAL":
        th_rmse = mean([(s["theta"] - s["pred_theta"].get_weights()[0][0])**2 for s in psi_list])
        b_rmse = mean([(q.beta - q.pred_beta.get_weights()[0][0])**2 for q in master_qs])
    else:
        th_rmse = mean([(s["theta"] - s["pred_theta_sc"])**2 for s in psi_list])
        b_rmse = mean([(q.beta - q.pred_beta_sc)**2 for q in master_qs])

    if (last_b_rmse - b_rmse) > b_del_tol:
        last_b_rmse = b_rmse
        print("b mse:", b_rmse, "del:", (last_b_rmse - b_rmse))
    else:
        freeze_betas = True

    if (last_th_rmse - th_rmse) > th_del_tol:
        last_th_rmse = th_rmse
        input("th mse: {} del: {}".format(th_rmse,(last_th_rmse - th_rmse)))
    else:
        freeze_thetas = True
 
    if freeze_betas and freeze_thetas: # NO MORE CHANGE POSSIBLE!
        break

print("final beta rmse = {}".format(b_rmse))
print("final theta rmse = {}".format(th_rmse))
        

sNOCHA GAG : true(0), [ 2.99848032] -> [ 0.30300346]
sTYHAD LOS : true(1), [ 3.27997017] -> [ 1.98007834]
sYYNY GEKETH : true(0), [ 0.24511337] -> [ 0.48933557]
sTHICH FETHAT : true(0), [ 3.71522617] -> [ 0.25398719]
sTHECHY YAVATH : true(4), [ 3.4565835] -> [ 3.75649214]
sWEJIC THON : true(4), [ 2.9414258] -> [ 3.92742848]
sDEW BEG : true(4), [ 2.40742636] -> [ 3.83896303]
sBYKUPH GYD : true(0), [ 3.78061199] -> [ 0.]
sLUN DYH : true(3), [ 2.33408737] -> [ 3.99374747]
sWOWYCH WINUB : true(1), [ 2.8631711] -> [ 1.911273]
sGIHYN KAW : true(2), [ 1.88300085] -> [ 1.34014785]
sHAGO THITHYR : true(2), [ 2.18468475] -> [ 2.44349599]
sVALA FIVOT : true(0), [ 3.88628435] -> [ 1.0359534]
sSEGAPH HOJ : true(3), [ 3.03456974] -> [ 2.75887609]
sBOPHIF SUYI : true(0), [ 0.05613613] -> [ 1.30416965]
sVUF SAHEF : true(4), [ 1.73440981] -> [ 3.24092293]
sCYLA KOB : true(3), [ 2.78973103] -> [ 2.27006555]
sCEJEB CUNI : true(2), [ 3.50475836] -> [ 2.77997351]
sWIT LOC : true(4), [ 2.6069541] -> [ 3.794

sKATHA TONEB : true(1), [ 2.41143179] -> [ 1.30639946]
sDUTYJ BOFY : true(0), [ 0.13419819] -> [ 0.16426775]
sCHACH BIDUR : true(0), [ 3.55121613] -> [ 0.13354403]
sTHICHIPH DOKU : true(2), [ 0.44915962] -> [ 3.28451538]
sMISE VUYE : true(1), [ 2.73078632] -> [ 1.75895929]
sCHIK TOFIM : true(0), [ 1.9363265] -> [ 1.50520396]
sJYC THOBOC : true(1), [ 1.23019457] -> [ 1.2321074]
sTOCH NASO : true(0), [ 0.38252687] -> [ 1.13338923]
sRIH PHEHYK : true(3), [ 1.40516663] -> [ 3.10773301]
sDAM BOCH : true(3), [ 1.38285732] -> [ 1.81491649]
sJOCHO BOW : true(3), [ 1.83260155] -> [ 3.24718833]
sCHYG PAK : true(1), [ 0.2446146] -> [ 0.]
sCHOLA HULU : true(4), [ 1.50923252] -> [ 4.]
sVOYUC FAJ : true(4), [ 2.32827425] -> [ 4.]
sNYD KINO : true(2), [ 1.95674467] -> [ 2.36545253]
sKAGE THYPY : true(4), [ 1.14991426] -> [ 4.]
sWISUP FAD : true(3), [ 0.03561306] -> [ 4.]
sKAFIN JOSIH : true(1), [ 2.0550046] -> [ 1.31257606]
sTESI GAHAN : true(4), [ 3.04157782] -> [ 3.70168495]
sTHEV WIPH : true(3), [

In [16]:
mse_ths = []
for psi in psi_list:
    real_th = psi["theta"]
    if calibration_mode=="NEURAL":
        pred_th = psi["pred_theta"].get_weights()[0][0]
    else:
        pred_th = psi["pred_theta_sc"]        
    mse_ths.append( (real_th - pred_th)**2 )
    print(psi["name"],":",real_th,"vs",pred_th)
print("MSE in theta: {}".format(mean(mse_ths)))

mse_bs = []
for q in master_qs:
    real_b = q.beta
    if calibration_mode=="NEURAL":
        pred_b = q.pred_beta.get_weights()[0][0]
    else:
        pred_b = q.pred_beta_sc
    mse_bs.append( (real_b - pred_b)**2 )
    print(real_b, "vs", pred_b)
print("MSE in beta: {}".format(mean(mse_bs)))


CAWED CUTO  : 0 vs [ 1.43137002]
THAS LAMA  : 2 vs [ 3.13842344]
VOS NOPHE  : 0 vs [ 0.01624745]
TYGEPH CHAG  : 3 vs [ 2.69254637]
MAY NUBON  : 4 vs [ 2.65961027]
PHECHA VOD  : 2 vs [ 1.85923064]
LOBAN SUWU  : 2 vs [ 1.30683517]
DUMEK CHYCUW  : 4 vs [ 4.]
HAGO THITHYR  : 2 vs [ 0.09910488]
WOTH YAC  : 0 vs [ 2.53139877]
YIGE HUF  : 3 vs [ 3.54270101]
LOF CAP  : 0 vs [ 1.65312004]
BONIF GEPO  : 0 vs [ 1.52775323]
SEV JIDA  : 4 vs [ 1.1993494]
YONOS SIPHI  : 0 vs [ 0.38031149]
CEJAN HIGEPH  : 2 vs [ 3.43062305]
WUTETH VURYB  : 1 vs [ 3.92609978]
CEBOY MYF  : 1 vs [ 1.54486418]
HUMAK KOWO  : 2 vs [ 3.29947162]
SAC TUB  : 2 vs [ 3.22714806]
GETHOC CHUGY  : 4 vs [ 2.77694082]
LYTH GYCHAK  : 0 vs [ 0.90867782]
KEL NEP  : 4 vs [ 0.72990131]
DEDO SITHEG  : 0 vs [ 3.00751114]
WIFYK BOFETH  : 0 vs [ 3.28139877]
KOC TECH  : 4 vs [ 1.17192125]
CHUGO KOLITH  : 4 vs [ 3.13905096]
CEVOG PHAPHI  : 3 vs [ 3.67294765]
DAL SUN  : 2 vs [ 2.60088944]
RYKA NETU  : 3 vs [ 3.1335206]
RULY MOG  : 1 vs [ 0.9617

MECYB RYVAW  : 3 vs [ 2.23943806]
VEL SODO  : 1 vs [ 2.97607565]
BOPHIF SUYI  : 0 vs [ 3.1383996]
FIPHI PHUP  : 3 vs [ 0.28475714]
HYCHYM CAPH  : 3 vs [ 1.28557491]
MOPHEC JAGU  : 4 vs [ 2.16000319]
SOYIM YEM  : 4 vs [ 0.41595507]
COCHA YOJOL  : 0 vs [ 0.99709755]
THIRE COTH  : 4 vs [ 4.]
HUDO DOVY  : 3 vs [ 2.68534541]
SOFY THYFYP  : 2 vs [ 3.75350189]
WIYA THEPY  : 2 vs [ 3.10501432]
YIK SUPHI  : 2 vs [ 2.72359896]
JALU PHIHI  : 2 vs [ 3.25909042]
RUCHOK FICEP  : 4 vs [ 1.9119544]
DOPH CULYS  : 1 vs [ 0.78467852]
NYD KINO  : 2 vs [ 0.5337925]
NAD PUH  : 4 vs [ 3.98614693]
NEJO TIBY  : 2 vs [ 3.70726848]
SEWE GUMOD  : 0 vs [ 0.27879572]
SYR ROM  : 0 vs [ 0.12285166]
DICI TYJO  : 4 vs [ 0.0578723]
PUTHOY CAPHU  : 0 vs [ 1.91644573]
PAWIL TAVO  : 2 vs [ 2.24666524]
KAVYN YAGUY  : 1 vs [ 0.49460697]
FAME MIG  : 4 vs [ 3.5415802]
TENAT HINYC  : 4 vs [ 3.66165328]
POTH PHAHU  : 1 vs [ 0.12483168]
TYMOW YYNU  : 2 vs [ 3.85567951]
NUCHED HATH  : 0 vs [ 0.82624865]
PHAMIPH KISA  : 2 vs [ 1.83

PIWOK PAHA  : 3 vs [ 2.18360853]
CHIF MEPHY  : 4 vs [ 1.84875488]
FIR PHIDE  : 0 vs [ 2.39810562]
PHYG THANES  : 0 vs [ 1.95337915]
ROPHU KUCH  : 1 vs [ 0.84565866]
CHACHYN NOS  : 1 vs [ 2.10479641]
KAFIN JOSIH  : 1 vs [ 1.0605979]
VALA FIVOT  : 0 vs [ 1.27124071]
SEGAPH HOJ  : 3 vs [ 2.73494864]
WIFE BIYE  : 4 vs [ 3.95759606]
MOBOH LIJ  : 3 vs [ 1.21000147]
WOBI YIPHYM  : 4 vs [ 2.68863821]
NIBIR PHIFAS  : 3 vs [ 3.62721419]
HAPHIT HUBUP  : 0 vs [ 0.18194611]
PHICHU RAJ  : 1 vs [ 1.57024658]
RUFO PHAGAY  : 3 vs [ 3.73447585]
FIGO KOJO  : 4 vs [ 4.]
MOMIP VIHYCH  : 0 vs [ 0.92615455]
MALOK TIKAV  : 0 vs [ 0.81912708]
RETU DOT  : 4 vs [ 2.80849028]
THYRYJ JYK  : 1 vs [ 0.75340223]
SENOD PYH  : 0 vs [ 0.88834316]
DIBIPH DUHE  : 3 vs [ 2.42515564]
SYFUG HIHI  : 4 vs [ 4.]
TUW SETIL  : 3 vs [ 3.23864627]
YAMO JIRO  : 1 vs [ 0.3250885]
JASAG WIY  : 1 vs [ 1.98364735]
FYY JYTO  : 4 vs [ 3.16597939]
CIBE YID  : 1 vs [ 3.7421031]
PEPHOF JOHA  : 2 vs [ 2.77743769]
WIWY WYPH  : 2 vs [ 2.7947108

CHIYYS YIT  : 0 vs [ 3.63691854]
GYPHY CHYD  : 2 vs [ 2.8205843]
MOF GOMA  : 4 vs [ 0.71576881]
GEBOM DOCO  : 4 vs [ 3.57227087]
MIC LASAW  : 1 vs [ 3.8584671]
GICHOH CHOVAH  : 1 vs [ 0.90113705]
PHYMO MACO  : 3 vs [ 1.38056564]
PHACHE PHETAK  : 1 vs [ 0.90177459]
NIKUG KUVI  : 2 vs [ 3.78133821]
SAS DISU  : 2 vs [ 1.4956913]
JESA KEPA  : 4 vs [ 4.]
CAVY CIM  : 2 vs [ 1.35585976]
NAPH BUTOB  : 3 vs [ 3.52347565]
VYVI YAHY  : 2 vs [ 2.31182599]
YYP PER  : 0 vs [ 2.39385319]
DASI TYLOTH  : 2 vs [ 2.7417345]
PHOB COR  : 1 vs [ 1.37065291]
YET MOYIR  : 3 vs [ 3.10119891]
TUV KUTHON  : 4 vs [ 4.]
PUB CHAYE  : 0 vs [ 2.0533123]
HEP KEC  : 3 vs [ 3.22403097]
TIM CAPA  : 0 vs [ 0.99424171]
BYC YAW  : 1 vs [ 1.99227655]
HAPHO SADO  : 3 vs [ 2.86446309]
SIMOF BADU  : 3 vs [ 2.85352731]
SICH PHAF  : 0 vs [ 3.74575281]
FYMA JAD  : 0 vs [ 1.46181238]
FONO VOM  : 1 vs [ 3.44973373]
PAFIW PHUKOY  : 1 vs [ 1.53738427]
RAYO CHES  : 1 vs [ 0.67450428]
PYNU CHERO  : 4 vs [ 2.28033829]
MUVA ROGEN  : 0 vs 